In [1]:
import random
import numpy

## 주사위

In [2]:
def dice():
    return random.randint(1, 3)

In [3]:
dice()

1

In [6]:
dice()

3

## 미니 블랙잭

In [7]:
def mini_blackjack(policy):
    dealer = dice()
    print('dealer', dealer)
    dealer_go = True
    
    player = dice()
    print('player', player)
    player_go = True
    
    while dealer_go or player_go:
        
        # 딜러는 3점 미만이면 무조건 주사위 굴림
        dealer_go = dealer_go and dealer < 3
        if dealer_go:
            dealer += dice()
            print('dealer', dealer)
            if dealer > 4:  # 4점을 넘어가면 버스트
                return 1    # 승리
            
        # 플레이어는 정책이 1이면 주사위 굴림, 아니면 패스   
        player_go = player_go and policy[player - 1, dealer - 1]
        if player_go:  
            player += dice()
            print('player', player)
            if player > 4:  # 4점을 넘어가면 버스트
                return -1   # 패배

    if player > dealer:
        return 1  # 승리
    elif player == dealer:
        return 0  # 무승부
    else:
        return -1 # 패배

## 정책

In [12]:
policy = numpy.array([
        [1, 1, 1, 1], # 1점일때는 남이 무슨 점수거든 고 
        [1, 1, 1, 1],
        [0, 0, 0, 0],
        [0, 0, 0, 0]
    ])

In [13]:
total = 0
for i in range(10):
    print('game {}'.format(i))
    ret = mini_blackjack(policy)
    print('return: {}'.format(ret))
    print()
    total += ret
print(total / 10)

game 0
dealer 1
player 2
dealer 3
player 4
return: 1

game 1
dealer 3
player 1
player 2
player 4
return: 1

game 2
dealer 1
player 2
dealer 4
player 3
return: -1

game 3
dealer 2
player 2
dealer 3
player 5
return: -1

game 4
dealer 3
player 3
return: 0

game 5
dealer 2
player 2
dealer 3
player 5
return: -1

game 6
dealer 1
player 2
dealer 2
player 5
return: -1

game 7
dealer 2
player 1
dealer 4
player 3
return: -1

game 8
dealer 3
player 3
return: 0

game 9
dealer 1
player 1
dealer 2
player 4
dealer 4
return: 0

-0.3


In [14]:
def blackjack(policy):
    episode = []
    dealer = dice()
    dealer_go = True
    
    player = dice()
    player_go = True
    
    while dealer_go or player_go:
        
        # 딜러는 3점 미만이면 무조건 주사위 굴림
        dealer_go = dealer_go and dealer < 3
        
        # 플레이어는 정책이 1이면 주사위 굴림, 아니면 패스   
        player_go = player_go and policy[player - 1, dealer - 1]
        episode.append((player, dealer, int(player_go)))        
        
        if dealer_go:
            dealer += dice()
        
        if player_go:  
            player += dice()

        # 4점이 넘어가면 버스트
        if player > 4 and dealer > 4:
            return 0, episode  # 무승부
        elif dealer > 4:
            return 1, episode  # 승리
        elif player > 4:
            return -1, episode  # 패배
        # 그 외의 경우에는 계속

    if player > dealer:
        result = 1  # 승리
    elif player == dealer:
        result = 0  # 무승부
    else:
        result = -1 # 패배
    return result, episode 

In [15]:
blackjack(policy)

(0, [(3, 2, 0), (3, 3, 0)])

## 몬테 카를로 법

In [16]:
returns = numpy.zeros((4, 4, 2)) # 4,4 하거나 안하거나 하니깐 2 

In [17]:
returns

array([[[ 0.,  0.],
        [ 0.,  0.],
        [ 0.,  0.],
        [ 0.,  0.]],

       [[ 0.,  0.],
        [ 0.,  0.],
        [ 0.,  0.],
        [ 0.,  0.]],

       [[ 0.,  0.],
        [ 0.,  0.],
        [ 0.,  0.],
        [ 0.,  0.]],

       [[ 0.,  0.],
        [ 0.,  0.],
        [ 0.,  0.],
        [ 0.,  0.]]])

## 특정한 상태를 거친 횟수

In [18]:
count = numpy.zeros((4, 4, 2))
count

array([[[ 0.,  0.],
        [ 0.,  0.],
        [ 0.,  0.],
        [ 0.,  0.]],

       [[ 0.,  0.],
        [ 0.,  0.],
        [ 0.,  0.],
        [ 0.,  0.]],

       [[ 0.,  0.],
        [ 0.,  0.],
        [ 0.,  0.],
        [ 0.,  0.]],

       [[ 0.,  0.],
        [ 0.,  0.],
        [ 0.,  0.],
        [ 0.,  0.]]])

## 시작정책 

In [19]:
policy = numpy.array([
        [1, 1, 1, 1],
        [0, 0, 0, 0],
        [0, 0, 0, 0],
        [0, 0, 0, 0]
    ])

## 에피소드마다 return과 횟수를 누적

In [20]:
reward = 0
discount = 1

In [21]:
for _ in range(1000):
    ret, episode = blackjack(policy)
    for player, dealer, action in reversed(episode):
        count[player - 1, dealer - 1, action] += 1
        returns[player - 1, dealer - 1, action] += ret
        ret = reward + discount * ret

## 결과 

In [22]:
returns

array([[[   0.,  -22.],
        [   0.,    2.],
        [   0.,    4.],
        [   0.,    0.]],

       [[ -81.,    0.],
        [ -17.,    0.],
        [-281.,    0.],
        [ -87.,    0.]],

       [[ -32.,    0.],
        [   7.,    0.],
        [   0.,    0.],
        [-107.,    0.]],

       [[   0.,    0.],
        [   5.,    0.],
        [  70.,    0.],
        [   0.,    0.]]])

In [23]:
count

array([[[   0.,   94.],
        [   0.,  113.],
        [   0.,  115.],
        [   0.,    0.]],

       [[ 107.,    0.],
        [ 147.,    0.],
        [ 281.,    0.],
        [  87.,    0.]],

       [[ 113.,    0.],
        [ 158.,    0.],
        [ 274.,    0.],
        [ 107.,    0.]],

       [[   0.,    0.],
        [  10.,    0.],
        [  70.,    0.],
        [  26.,    0.]]])

## 가치(= return의 평균) 계산

In [24]:
returns / count

C:\Anaconda3\envs\py35\lib\site-packages\ipykernel\__main__.py:1: RuntimeWarning: invalid value encountered in true_divide
  if __name__ == '__main__':


array([[[        nan, -0.23404255],
        [        nan,  0.01769912],
        [        nan,  0.03478261],
        [        nan,         nan]],

       [[-0.75700935,         nan],
        [-0.11564626,         nan],
        [-1.        ,         nan],
        [-1.        ,         nan]],

       [[-0.28318584,         nan],
        [ 0.0443038 ,         nan],
        [ 0.        ,         nan],
        [-1.        ,         nan]],

       [[        nan,         nan],
        [ 0.5       ,         nan],
        [ 1.        ,         nan],
        [ 0.        ,         nan]]])

## 에러 메시지를 막음

In [25]:
with numpy.errstate(divide='ignore', invalid='ignore'):
    value = returns / count

In [26]:
value

array([[[        nan, -0.23404255],
        [        nan,  0.01769912],
        [        nan,  0.03478261],
        [        nan,         nan]],

       [[-0.75700935,         nan],
        [-0.11564626,         nan],
        [-1.        ,         nan],
        [-1.        ,         nan]],

       [[-0.28318584,         nan],
        [ 0.0443038 ,         nan],
        [ 0.        ,         nan],
        [-1.        ,         nan]],

       [[        nan,         nan],
        [ 0.5       ,         nan],
        [ 1.        ,         nan],
        [ 0.        ,         nan]]])

In [28]:
~numpy.isfinite(value)

array([[[ True, False],
        [ True, False],
        [ True, False],
        [ True,  True]],

       [[False,  True],
        [False,  True],
        [False,  True],
        [False,  True]],

       [[False,  True],
        [False,  True],
        [False,  True],
        [False,  True]],

       [[ True,  True],
        [False,  True],
        [False,  True],
        [False,  True]]], dtype=bool)

In [29]:
value[~numpy.isfinite(value)] = 0

In [30]:
value[2, 3, 0]  # 플레이어가 3점, 딜러가 4점일 때 스톱!의 가치

-1.0

In [31]:
value[2, 3, 1]  # 플레이어가 3점, 딜러가 4점일 때 고!의 가치

0.0

## Policy Iteration

In [32]:
def compute_value(returns, count):
    with numpy.errstate(divide='ignore', invalid='ignore'):
        value = returns / count
    value[~numpy.isfinite(value)] = 0
    return value

In [33]:
policy = numpy.array([
        [1, 1, 1, 1],
        [0, 0, 0, 0],
        [0, 0, 0, 0],
        [0, 0, 0, 0]
    ])
returns = numpy.ones((4, 4, 2)) * 10  # 낙관적 초기화
count = numpy.zeros((4, 4, 2))

In [34]:

total = 0
for i in range(1000):
    ret, _ = blackjack(policy)
    total += ret
print(total / 1000)

-0.229


In [35]:
for _ in range(1000):
    ret, episode = blackjack(policy)
    for player, dealer, action in reversed(episode):
        count[player - 1, dealer - 1, action] += 1
        returns[player - 1, dealer - 1, action] += ret
        ret = reward + discount * ret
        
    ## 가치 수정
    value = compute_value(returns, count)
        
    ## 정책 수정
    for p in range(4):
        for d in range(4):
            policy[p, d] = numpy.argmax(value[p, d])

In [36]:
policy

array([[1, 1, 1, 0],
       [1, 1, 1, 1],
       [1, 0, 0, 1],
       [0, 0, 0, 0]])

In [37]:
total = 0
for i in range(1000):
    ret, _ = blackjack(policy)
    total += ret
print(total / 1000)

0.014


In [38]:
value

array([[[ 0.        ,  0.10687023],
        [-0.04347826,  0.21276596],
        [-0.28571429,  0.37837838],
        [ 0.        ,  0.        ]],

       [[-0.05882353,  0.11904762],
        [-0.03448276,  0.07476636],
        [-0.56521739,  0.01162791],
        [-0.28571429, -0.25      ]],

       [[-0.22222222, -0.22      ],
        [-0.01123596, -0.03389831],
        [ 0.03496503,  0.        ],
        [-0.84848485, -0.55357143]],

       [[ 0.        ,  0.        ],
        [ 0.97142857,  0.        ],
        [ 1.06134969,  0.        ],
        [ 0.12658228,  0.        ]]])

## 시간차 학습

In [39]:

policy = numpy.array([
        [1, 1, 1, 1],
        [0, 0, 0, 0],
        [0, 0, 0, 0],
        [0, 0, 0, 0]
    ])
value = numpy.zeros((4, 4, 2))  # return 대신 value를 사용
count = numpy.zeros((4, 4, 2))

In [40]:
for _ in range(1000):
    ret, episode = blackjack(policy)
    next_value = 0
    for i, state_action in enumerate(reversed(episode)):
        player, dealer, action = state_action
        if i == 0:
            reward = ret
        else:
            reward = 0
        p = player - 1
        d = dealer - 1
        count[p, d, action] += 1
        
        # 가치 수정
        N = count[p, d, action] 
        current_value = value[p, d, action]
        bellman = reward + discount * next_value  # 벨만 방정식
        next_value = value[p, d, action] = current_value + (bellman - current_value) / N  # 이동 평균

In [41]:
value

array([[[ 0.        , -0.15930073],
        [ 0.        , -0.06060606],
        [ 0.        ,  0.00840336],
        [ 0.        ,  0.        ]],

       [[-0.75941207,  0.        ],
        [-0.32571429,  0.        ],
        [-1.        ,  0.        ],
        [-1.        ,  0.        ]],

       [[-0.39353533,  0.        ],
        [-0.06918239,  0.        ],
        [ 0.        ,  0.        ],
        [-1.        ,  0.        ]],

       [[ 0.        ,  0.        ],
        [ 0.75      ,  0.        ],
        [ 1.        ,  0.        ],
        [ 0.        ,  0.        ]]])

## SARSA